In [1]:
val replClassPathObj = os.Path("replClasses", os.pwd)
if (!os.exists(replClassPathObj)) os.makeDir(replClassPathObj)
val replClassPath = replClassPathObj.toString()

replClassPathObj: os.Path = /home/jovyan/replClasses
replClassPath: String = "/home/jovyan/replClasses"

In [2]:
interp.configureCompiler(_.settings.outputDirs.setSingleOutput(replClassPath))
interp.configureCompiler(_.settings.Yreplclassbased)
interp.load.cp(replClassPathObj)

In [3]:
// Import the Snowpark library from Maven.
import $ivy.`com.snowflake:snowpark:1.1.0`
import $ivy.`org.apache.logging.log4j:log4j-core:2.17.1`

import com.snowflake.snowpark._
import com.snowflake.snowpark.functions._

import com.snowflake.snowpark.functions.{col, lit}
import com.snowflake.snowpark.{DataFrame, RelationalGroupedDataFrame, Session}

import java.io.{File, InputStream}
import java.nio.file.Paths
import java.util.Properties
import scala.collection.JavaConverters._
import scala.collection.mutable

Downloaded https://repo1.maven.org/maven2/com/snowflake/snowpark/1.1.0/snowpark-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.pom
Downloaded https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.13.10/snowflake-jdbc-3.13.10.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.32/slf4j-api-1.7.32.pom
Downloaded https://repo1.maven.org/maven2/javax/xml/bind/jaxb-api/2.2.2/jaxb-api-2.2.2.pom
Downloaded https://repo1.maven.org/maven2/commons-codec/commons-codec/1.15/commons-codec-1.15.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-simple/1.7.32/slf4j-simple-1.7.32.pom
Downloaded https://repo1.maven.org/maven2/com/github/vertical-blank/sql-formatter/1.0.2/sql-formatter-1.0.2.pom
Downloaded https://repo1.maven.org/maven2/org/junit/junit-bom/5.4.2/junit-bom-5.4.2.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.32/slf4j-parent-1.7.32.pom
Downloaded https://repo1.maven.org/maven

import $ivy.$                             

import $ivy.$                                           


import com.snowflake.snowpark._

import com.snowflake.snowpark.functions._


import com.snowflake.snowpark.functions.{col, lit}

import com.snowflake.snowpark.{DataFrame, RelationalGroupedDataFrame, Session}


import java.io.{File, InputStream}

import java.nio.file.Paths

import java.util.Properties

import scala.collection.JavaConverters._

import scala.collection.mutable

In [4]:

import org.apache.logging.log4j.core.config.Configurator;
import org.apache.logging.log4j.Level;
Configurator.setRootLevel(Level.WARN);



import org.apache.logging.log4j.core.config.Configurator;

import org.apache.logging.log4j.Level;


In [5]:
    val session = Session.builder.configs(Map(
    "URL" -> "",
    "USER" -> "",
    "PASSWORD" -> "",
    "ROLE" -> "",
    "WAREHOUSE" -> "",
    "DB" -> "",
    "SCHEMA" -> ""
)).create

[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Closing stderr and redirecting to stdout


[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Done closing stderr and redirecting to stdout
[scala-interpreter-1] INFO com.snowflake.snowpark.internal.ParameterUtils - set JDBC client memory limit to 10240
[scala-interpreter-1] INFO com.snowflake.snowpark.Session - Snowpark Session information: {
 "snowpark.version" : "1.1.0",
 "java.version" : "1.8.0_265",
 "scala.version" : "2.12.12",
 "jdbc.session.id" : "95902324822282",
 "os.name" : "Linux",
 "jdbc.version" : "3.13.10",
 "snowpark.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/snowflake/snowpark/1.1.0/snowpark-1.1.0.jar",
 "scala.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.12/scala-library-2.12.12.jar",
 "jdbc.library" : "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.13.10/snowflake-jdbc-3.13.10.jar"
}


session: Session = com.snowflake.snowpark.Session@6b59a24c

In [6]:
  def createDF(tableName : String) : DataFrame ={
      val df: DataFrame = session.table(tableName)
      df
  }

  def runProcess(tableName : String): Unit ={
    val df: DataFrame = session.table(tableName)
    val dfs = mutable.ArrayBuffer[DataFrame]()
    val fieldCount = df.schema.fields.size
    var iterCount = 0
    println(s"Total records in table : "
      + df.count().toString)
    df.schema.fields.foreach(i => {
      iterCount = iterCount + 1
      println(s"-- Processing column ${iterCount}/${fieldCount}: ${i.name}")
      println(s"Datatype : "+i.dataType.toString())
      val colAnalysis = createAnalysisDataframe(session,tableName, i.name, i.dataType.toString())
      dfs += colAnalysis
    })
    val analysisOutput = dfs.reduce(_ union _)
    analysisOutput.show(fieldCount)
    analysisOutput.write.mode("overwrite").saveAsTable(s"SNOWPARK_DATA_PROFILING_TPCDS_STORE")
  }


  // Rules definition
  def colValueDuplicates(df: DataFrame, columnName: String): String = {
    val dataframe1 = df.groupBy(columnName).count()
    val dataframe2 = dataframe1.filter(col("count") === 1)
    val duplicates = (dataframe1.count() - dataframe2.count()).toString
    println(s"No of duplicate records in column : ${duplicates}")
    duplicates
  }

  def colValueUniqueness(df: DataFrame, columnName: String): Int = {
    val dataframe1 = df.groupBy(col(columnName)).count()
    val dataframe2 = dataframe1.filter(col("count") === 1).count()

    println(s"No of unique values in column : "
      + dataframe2.toString)
    dataframe2.toInt
  }

  def colNullCheck(df: DataFrame, columnName: String): String = {
    val nulls = df.where(df.col(columnName).is_null).count()
    println(s"No of null values in column : "
      + nulls.toString)
    nulls.toString
  }

  def colValueCompleteness(df: DataFrame, columnName: String): String = {
    val nulls : Double = df.where(df.col(columnName).is_null).count()
    val count : Double = df.count().toDouble
    val completeness = 1-(nulls / count)
    println(s"Completeness of column : "
      + completeness.toString)
    completeness.toString
  }

  def colStatistics(df: DataFrame, columnName: String, dataType : String): String = {
    var stats = ""
    if (dataType != "String" && dataType != "Date") {
      val max = df.select(functions.max(df.col(columnName))).collect()(0)(0)
      val min = df.select(functions.min(df.col(columnName))).collect()(0)(0)
      val stdev = df.select(functions.stddev(df.col(columnName))).collect()(0)(0)
      val mean = df.select(functions.mean(df.col(columnName))).collect()(0)(0)
      stats = min + "/" + max + "/" + stdev + "/" + mean
      println(s"Statistics of column : " + stats)
      stats
    } else{
      stats
    }
  }

  def colDatatype(df: DataFrame, columnName: String): String = {
    val nulls : Double = df.where(df.col(columnName).is_null).count()
    //    val completeness = if (nulls>0) nulls/df.count() else 1.0
    val count : Double = df.count().toDouble
    val completeness = 1-(nulls / count)
    println(s"Completeness of column : " + completeness.toString)
    completeness.toString
  }

  def createAnalysisDataframe(session: Session, tableName : String, columnName : String, dataType : String): DataFrame = {
    val df: DataFrame = session.table(tableName)
    val dfSeq = session.createDataFrame(Seq(columnName)).toDF("columnName")
    val dfOut = dfSeq.withColumn("completeness", lit(colValueCompleteness(df, columnName)))
      .withColumn("uniqueValues", lit(colValueUniqueness(df, columnName)))
      .withColumn("duplicates",lit(colValueDuplicates(df, columnName)))
      .withColumn("nulls",lit(colNullCheck(df, columnName)))
      .withColumn("datatype",lit(dataType))
      .withColumn("MIN_MAX_MEAN_STDEV",lit(colStatistics(df, columnName, dataType)))
    dfOut
  }

  def createPermanentUdf(session: Session): Unit = {
    session.sql("drop FUNCTION if EXISTS udfPerm(Int)").show()
    session.udf.registerPermanent("udfPerm", permanentUdfHandler _, "STG")
    session.sql("select *, udfPerm(*) from values (10)").show()
  }

  def permanentUdfHandler(i: Int): Int = {
    i * i
  }

  def createInlinePermanentUdf(session: Session): Unit = {
    session.sql("drop FUNCTION if EXISTS udfInlinePerm(Int)").show()

    session.udf.registerPermanent("udfInlinePerm", (i: Int) => {
      i * 2
    }, "STG")

    session.sql("select *, udfInlinePerm(*) from values (10)").show()
  }

  private def addDeps(session: Session): Unit = {
    val PATH = Paths.get(".", "target", "dependency").toAbsolutePath.toString
    val lst = getListOfFiles(PATH)
    val filteredLst = lst.filterNot(_.matches("^.*snowpark(_original)?-[0-9.]+\\.jar$"))
    for (f <- filteredLst) {
      System.out.println("Adding dep:" + f)
      session.addDependency(f)
    }
  }

  def getListOfFiles(dir: String): List[String] = {
    val d = new File(dir)
    if (d.exists && d.isDirectory) {
      d.listFiles.filter(_.isFile).toList.map(_.getPath)
    } else {
      List[String]()
    }
  }
   

defined function createDF
defined function runProcess
defined function colValueDuplicates
defined function colValueUniqueness
defined function colNullCheck
defined function colValueCompleteness
defined function colStatistics
defined function colDatatype
defined function createAnalysisDataframe
defined function createPermanentUdf
defined function permanentUdfHandler
defined function createInlinePermanentUdf
defined function getListOfFiles

In [7]:
val df = createDF("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.STORE")
df.schema.fields.foreach(i => {println(i)})

StructField(S_STORE_SK, Long, Nullable = false)
StructField(S_STORE_ID, String, Nullable = false)
StructField(S_REC_START_DATE, Date, Nullable = true)
StructField(S_REC_END_DATE, Date, Nullable = true)
StructField(S_CLOSED_DATE_SK, Long, Nullable = true)
StructField(S_STORE_NAME, String, Nullable = true)
StructField(S_NUMBER_EMPLOYEES, Long, Nullable = true)
StructField(S_FLOOR_SPACE, Long, Nullable = true)
StructField(S_HOURS, String, Nullable = true)
StructField(S_MANAGER, String, Nullable = true)
StructField(S_MARKET_ID, Long, Nullable = true)
StructField(S_GEOGRAPHY_CLASS, String, Nullable = true)
StructField(S_MARKET_DESC, String, Nullable = true)
StructField(S_MARKET_MANAGER, String, Nullable = true)
StructField(S_DIVISION_ID, Long, Nullable = true)
StructField(S_DIVISION_NAME, String, Nullable = true)
StructField(S_COMPANY_ID, Long, Nullable = true)
StructField(S_COMPANY_NAME, String, Nullable = true)
StructField(S_STREET_NUMBER, String, Nullable = true)
StructField(S_STREET_NAM

df: DataFrame = com.snowflake.snowpark.Updatable@14637357

In [8]:
runProcess("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.STORE")

Total records in table : 1902
-- Processing column 1/29: S_STORE_SK
Datatype : Long
Completeness of column : 1.0
No of unique values in column : 1902
No of duplicate records in column : 0
No of null values in column : 0
Statistics of column : 1/1902/549.2044245998023/951.500000
-- Processing column 2/29: S_STORE_ID
Datatype : String
Completeness of column : 1.0
No of unique values in column : 317
No of duplicate records in column : 634
No of null values in column : 0
-- Processing column 3/29: S_REC_START_DATE
Datatype : Date
Completeness of column : 0.9942166140904312
No of unique values in column : 0
No of duplicate records in column : 5
No of null values in column : 11
-- Processing column 4/29: S_REC_END_DATE
Datatype : Date
Completeness of column : 0.5
No of unique values in column : 0
No of duplicate records in column : 4
No of null values in column : 951
-- Processing column 5/29: S_CLOSED_DATE_SK
Datatype : Long
Completeness of column : 0.288117770767613
No of unique values in 